In [ ]:
!pip install sagemaker
!pip install boto3

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import boto3
import botocore
import sagemaker
import sys
from sagemaker import RandomCutForest
taxi_data = pd.read_csv('nyc_taxi.csv', delimiter=",")


In [ ]:

session = sagemaker.Session()

# RCF 모델 생성
rcf = RandomCutForest(
    role='sagemaker-execution-role',
    instance_count=1,
    instance_type="ml.m4.xlarge",
    data_location=f"s3://bucket/data/",
    output_path=f"s3://bucket/output",
    num_samples_per_tree=512,
    num_trees=50,
)

# automatically upload the training data to S3 and run the training job
rcf.fit(rcf.record_set(taxi_data.value.to_numpy().reshape(-1, 1)))

In [ ]:
print(f"Training job name: {rcf.latest_training_job.job_name}")

In [ ]:
# RCF 모델 배포
rcf_inference = rcf.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")

In [ ]:
print(f"Endpoint name: {rcf_inference.endpoint}")

In [ ]:
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

rcf_inference.serializer = CSVSerializer()
rcf_inference.deserializer = JSONDeserializer()

taxi_data_numpy = taxi_data.value.to_numpy().reshape(-1, 1)
print(taxi_data_numpy[:6])
results = rcf_inference.predict(
    taxi_data_numpy[:6], initial_args={"ContentType": "text/csv", "Accept": "application/json"}
)

In [ ]:
results = rcf_inference.predict(taxi_data_numpy)
scores = [datum["score"] for datum in results["scores"]]

taxi_data["score"] = pd.Series(scores, index=taxi_data.index)
taxi_data.head()

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

start, end = 0, len(taxi_data)
# start, end = 5500, 6500
taxi_data_subset = taxi_data[start:end]

ax1.plot(taxi_data_subset["value"], color="C0", alpha=0.8)
ax2.plot(taxi_data_subset["score"], color="C1")

ax1.grid(which="major", axis="both")

ax1.set_ylabel("Taxi Ridership", color="C0")
ax2.set_ylabel("Anomaly Score", color="C1")

ax1.tick_params("y", colors="C0")
ax2.tick_params("y", colors="C1")

ax1.set_ylim(0, 40000)
ax2.set_ylim(min(scores), 1.4 * max(scores))
fig.set_figwidth(10)

In [ ]:
score_mean = taxi_data["score"].mean()
score_std = taxi_data["score"].std()
score_cutoff = score_mean + 3 * score_std

anomalies = taxi_data_subset[taxi_data_subset["score"] > score_cutoff]
anomalies

In [ ]:
ax2.plot(anomalies.index, anomalies.score, "ko")
fig

In [ ]:
sagemaker.Session().delete_endpoint(rcf_inference.endpoint)